In [30]:
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED
import boto3
import json

### Retrieve data from S3

In [2]:
#Initialize S3 client or resource
s3_client = boto3.client('s3') 

In [33]:
#Data source 1 : pr.data.0.Current series data 
bucket_name = "rearc-takehome-bucket"
series_data_path = file_key = "pr.data.0.Current"

# Fetch the series file data into df
try :
    obj = s3_client.get_object(Bucket=bucket_name, Key=file_key)

    # Read into Pandas
    series_df = pd.read_csv(obj["Body"], sep="\t")

    # cleans the columns
    series_df.columns = series_df.columns.str.strip().str.lower()

    print(series_df.head())
    

except Exception as e:
    print(f"Error loading series data from S3: {e}")

           series_id  year period  value footnote_codes
0  PRS30006011        1995    Q01    2.6            NaN
1  PRS30006011        1995    Q02    2.1            NaN
2  PRS30006011        1995    Q03    0.9            NaN
3  PRS30006011        1995    Q04    0.1            NaN
4  PRS30006011        1995    Q05    1.4            NaN


In [34]:
# Load Data source 2 : population data

population_data_bucket = "honolulu-population-data"
population_data_file_key = "honolulu-population-data_20250824_170058.json"

# Get the object from S3
try:
    response = s3_client.get_object(Bucket=population_data_bucket, Key=population_data_file_key)
    
    # Read the content of the object body
    json_data = response['Body'].read().decode('utf-8')
    
    # Parse the JSON data and load into pandas df
    data = json.loads(json_data)
    population_df = pd.DataFrame(data['data'], columns=data['columns'] )
    population_df.columns = population_df.columns.str.strip().str.lower()

    print(population_df.head())


except Exception as e:
    print(f"Error loading JSON file from S3: {e}")

  nation id         nation  year   population
0   01000US  United States  2013  316128839.0
1   01000US  United States  2014  318857056.0
2   01000US  United States  2015  321418821.0
3   01000US  United States  2016  323127515.0
4   01000US  United States  2017  325719178.0


### Question - 1

#### Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive.

In [6]:
filtered_population_df = population_df[(population_df['year'].between(2013,2018)) & (population_df['nation']=='United States')]

In [7]:
#stats
mean_population = filtered_population_df["population"].mean()
std_population = filtered_population_df["population"].std()
mean_population, std_population

(322069808.0, 4158441.040908095)

### Question - 2: 

#### Using the dataframe from the time-series (Part 1),  For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year.


In [9]:
yearly_sum = series_df.groupby(["series_id", "year"], as_index=False)["value"].sum()

best = (yearly_sum.sort_values(["series_id", "value"], ascending=[True, False])
                  .drop_duplicates("series_id", keep="first")
                  .rename(columns={"year": "best_year", "value": "total_value"})
                  .reset_index(drop=True))
best

,series_id,best_year,total_value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,705.895
3,PRS30006021,2010,17.700
4,PRS30006022,2010,12.400
...,...,...,...
277,PRS88003192,2002,282.800
278,PRS88003193,2024,860.838
279,PRS88003201,2022,37.200
280,PRS88003202,2022,28.700


### Question - 3
#### Using both dataframes from Part 1 and Part 2, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset)  filtered_series_df = series_df.loc[(series_df['series_id'] == 'PRS30006032') & (series_df['period'] == 'Q01')]


In [17]:
# cleanup: normalize spaces, strip, UPPER
series_df["series_id_clean"] = (series_df["series_id"].astype(str)
                                  .str.replace(r"\s+", " ", regex=True)
                                  .str.strip()
                                  .str.upper())

series_df["period_clean"] = (series_df["period"].astype(str)
                               .str.replace(r"\s+", " ", regex=True)
                               .str.strip()
                               .str.upper())

# filter the series df with given specific conditions
filtered_series_df = series_df.loc[
    (series_df["series_id_clean"] == "PRS30006032") &
    (series_df["period_clean"] == "Q01")
][["series_id", "year", "period", "value"]]

filtered_series_df.head()

,series_id,year,period,value
1064,PRS30006032,1995,Q01,0.0
1069,PRS30006032,1996,Q01,-4.2
1074,PRS30006032,1997,Q01,2.8
1079,PRS30006032,1998,Q01,0.9
1084,PRS30006032,1999,Q01,-4.1


In [29]:
report = filtered_series_df.merge(population_df, on="year", how="left")

# Drop rows with missing population
report = report.dropna(subset=["population"])

# Final clean report
report = report[["series_id", "year", "period", "value", "population"]]

report

,series_id,year,period,value,population
18,PRS30006032,2013,Q01,0.5,316128839.0
19,PRS30006032,2014,Q01,-0.1,318857056.0
20,PRS30006032,2015,Q01,-1.7,321418821.0
21,PRS30006032,2016,Q01,-1.4,323127515.0
22,PRS30006032,2017,Q01,0.9,325719178.0
23,PRS30006032,2018,Q01,0.5,327167439.0
24,PRS30006032,2019,Q01,-1.6,328239523.0
26,PRS30006032,2021,Q01,0.7,331893745.0
27,PRS30006032,2022,Q01,5.3,333287562.0
28,PRS30006032,2023,Q01,0.3,334914896.0
